### 1) API call to get statewide GDOT projects


In [2]:
# depencencies
import requests
import pandas as pd
import geopandas as gpd
from datetime import datetime
import json
import numpy as np
import glob
import os

# Define the base URL of the MapServer layer
base_url = "https://rnhp.dot.ga.gov/hosting/rest/services/GDOT_Public_Outreach/Hub_Project_Search/MapServer/2/query"

# Define query parameters
params = {
    # filter by construction status to include both under construction and pre-construction
    "where": "CONSTRUTION_STATUS_DERIVED LIKE '%'",
    "outFields": "*",  # Get all fields
    "f": "geojson",  # Request data in GeoJSON format
    "returnGeometry": "true",  # Include geometry data
    "resultOffset": 0,  # Start from the first record
    "resultRecordCount": 1000  # Get 1000 records
}

# list to store each batch of GeoDataFrames
gdfs = []

while True:

    # Make the GET request
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()

        batch_gdf = gpd.GeoDataFrame.from_features(data["features"])

        # if no more data is returned, break the loop
        if batch_gdf.empty:
            break

        gdfs.append(batch_gdf)

        print(
            f"Fetched {len(batch_gdf)} records (offset: {params['resultOffset']})")

        # increase offset by the batch size to fetch the next chunk
        params["resultOffset"] += params["resultRecordCount"]

    else:
        print(
            f"❌ Error fetching data: {response.status_code} - {response.text}")
        break

# Filter out empty GeoDataFrames
gdfs = [gdf.dropna(axis=1, how='all') for gdf in gdfs if not gdf.empty]

# Combine all batches into a single GeoDataFrame
if gdfs:
    gdot_projects = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

else:
    print("❌ No data retrieved.")

# drop any columns that ONLY contain NaN values
gdot_projects = gdot_projects.dropna(axis=1, how="all")

# replace spaces in the 'CONSTRUTION_STATUS_DERIVED' column with hyphens
gdot_projects['CONSTRUTION_STATUS_DERIVED'] = gdot_projects['CONSTRUTION_STATUS_DERIVED'].str.replace(
    " ", "-")
gdot_projects['CONSTRUCTION_STATUS_DERIVED'] = gdot_projects['CONSTRUTION_STATUS_DERIVED'].str.strip()

# if the 'CONTRACT_DESCRIPTION' column is empty, replace it with the 'SHORT_DESCR' column
gdot_projects['CONTRACT_DESCRIPTION'] = gdot_projects['CONTRACT_DESCRIPTION'].fillna(
    gdot_projects['SHORT_DESCR'])

# define a list of sub-strings to keep in all caps; everything else in the 'SHORT_DESCR' column should be converted to title case
keep_all_caps = ["SR ", "CO ", "CR ", "US ", "RD", "SO ",
                 "CS ", "MI ", "SE ", "NE ", "SW ", "NW ",
                 "NS ", "CSX", "CCTV", "-TIA", " - TIA", "LCI", "GRTA"]

# convert to title case
gdot_projects['CONTRACT_DESCRIPTION'] = gdot_projects['CONTRACT_DESCRIPTION'].str.title()
for sub_str in keep_all_caps:
    gdot_projects['CONTRACT_DESCRIPTION'] = gdot_projects['CONTRACT_DESCRIPTION'].str.replace(
        sub_str.title(), sub_str)

# clean up some of the description sub-strings
gdot_projects['CONTRACT_DESCRIPTION'] = gdot_projects['CONTRACT_DESCRIPTION'].str.replace(
    "Brdg", "Bridge")
gdot_projects['CONTRACT_DESCRIPTION'] = gdot_projects['CONTRACT_DESCRIPTION'].str.replace(
    "Mtn ", "Mountain ")
gdot_projects['CONTRACT_DESCRIPTION'] = gdot_projects['CONTRACT_DESCRIPTION'].str.replace(
    "Phase Ii", "Phase II")
gdot_projects['CONTRACT_DESCRIPTION'] = gdot_projects['CONTRACT_DESCRIPTION'].str.replace(
    "Ph Ii", "Phase II")
gdot_projects['CONTRACT_DESCRIPTION'] = gdot_projects['CONTRACT_DESCRIPTION'].str.replace(
    "Phase IIi", "Phase III")
gdot_projects['CONTRACT_DESCRIPTION'] = gdot_projects['CONTRACT_DESCRIPTION'].str.replace(
    "Ph IIi", "Phase III")
gdot_projects['CONTRACT_DESCRIPTION'] = gdot_projects['CONTRACT_DESCRIPTION'].str.replace(
    "Phase Iv", "Phase IV")
gdot_projects['CONTRACT_DESCRIPTION'] = gdot_projects['CONTRACT_DESCRIPTION'].str.replace(
    "Phase Vi", "Phase VI")
gdot_projects['CONTRACT_DESCRIPTION'] = gdot_projects['CONTRACT_DESCRIPTION'].str.replace(
    "Resf ", "Resurface ")
gdot_projects['CONTRACT_DESCRIPTION'] = gdot_projects['CONTRACT_DESCRIPTION'].str.replace(
    "Rsrf ", "Resurface ")
gdot_projects['CONTRACT_DESCRIPTION'] = gdot_projects['CONTRACT_DESCRIPTION'].str.replace(
    "Shldr ", "Shoulder ")
gdot_projects['CONTRACT_DESCRIPTION'] = gdot_projects['CONTRACT_DESCRIPTION'].str.replace(
    "Recnst", "Reconstruction")
gdot_projects['CONTRACT_DESCRIPTION'] = gdot_projects['CONTRACT_DESCRIPTION'].str.replace(
    "Cnst ", "Construction ")
gdot_projects['CONTRACT_DESCRIPTION'] = gdot_projects['CONTRACT_DESCRIPTION'].str.replace(
    "Fm ", "From ")

# filter out records that have either 'WPH' (completed) or 'REJ' (rejected) in the 'CONST_STAT_CD' column
gdot_projects = gdot_projects[~gdot_projects['CONST_STAT_CD'].isin([
                                                                   'WPH', 'REJ'])]

# rename columns
gdot_projects = gdot_projects.rename(columns={
    'LAST_REFRESH_DTTM': 'Last_refresh',
    'PROJECT_COUNTIES': 'Project_counties',
    'PROJ_ID': 'Project_ID',
    'CONTRACT_DESCRIPTION': 'Project_description',
    'CONTRACTOR_NAME': 'Contractor_name',
    'IS_TIA_PROJECT': 'Is_TIA_project',
    'CONSTRUTION_STATUS_DERIVED': 'CONSTRUCTION_STATUS_DERIVED'
})

# drop unneeded columns
gdot_projects = gdot_projects.drop(columns=[
    'OBJECTID',
    'PRIORITY_CD',
    'SOURCE_OF_CONSTRUCTION_DATES',
    'CONTRACT_ID',
    'CONSTRUTION_STATUS_DERIVED_RSN',
    'PAYMENT_PERCENT_COMPLETE',
    'ESRI_OID',
    'SHORT_DESCR',
    'REC_STATUS',
    'LET_RESP_CD',
    'PRIORITY_CD_DESCR',
    'CONST_STAT_CD',
    'CONSTRUCTION_PERCENT_COMPLETE',
    'CURR_COMPLETION_DATE',
    'AWARD_DATE'
])

# create URL column
gdot_projects['Project_URL'] = 'https://www.dot.ga.gov/applications/geopi/Pages/Dashboard.aspx?ProjectId=' + \
    gdot_projects['Project_ID'].astype(str)

# Read in 2023 congressional districts
gdot_projects = gdot_projects.set_crs(epsg=4326)
districts = gpd.read_file(
    'data/congressional_districts/cdistricts.geojson')
districts = districts.to_crs(epsg=4326)
districts = districts[[
    'DISTRICT',
    'geometry'
]]

# spatial join projects to get congressional district
gdot_projects = gpd.sjoin(
    gdot_projects,
    districts,
    how='left',
    predicate='intersects'
).drop(columns=['index_right'])

# convert 'DISTRICT' column to integer
gdot_projects['DISTRICT'] = gdot_projects['DISTRICT'].astype(int)

# drop any duplicate columns
gdot_projects = gdot_projects.T.drop_duplicates().T

# export the GeoDataFrame to a CSV file
csv_export = gdot_projects.drop(columns=['geometry']).to_csv(
    'GDOT_export_initial.csv', index=False)

# export timestamp file to be inserted in <div> on frontend
current_date = datetime.now().strftime("%B %d, %Y")
with open("data/current_date.txt", "w") as f:
    f.write(current_date)

print(f"✅ {len(gdot_projects):,} projects exported with timestamp.")

Fetched 1000 records (offset: 0)
Fetched 1000 records (offset: 1000)
Fetched 1000 records (offset: 2000)
Fetched 1000 records (offset: 3000)
Fetched 1000 records (offset: 4000)
Fetched 1000 records (offset: 5000)
Fetched 7 records (offset: 6000)
✅ 4,009 projects exported with timestamp.


### 2) If necessary, scrape exported GDOT projects to get additional information


In [4]:
# using glob, read in all CSV files in the 'scraped' folder
csv_files = glob.glob('scraped/*.csv')

# create an empty list to store the dataframes
dfs = []

# loop through the CSV files and read them into a list of dataframes
for file in csv_files:
    dfs.append(pd.read_csv(file))

# concatenate the dataframes into a single dataframe
GDOT_master = pd.concat(dfs, ignore_index=True)

# drop duplicates based on 'Project_ID', 'Total_Cost', while keeping the first instance
GDOT_master = GDOT_master.drop_duplicates(
    subset=['Project_ID', 'Total_Cost'], keep='first')

# compare gdot_projects to GDOT_master to see if any new Project_ID values are present
gdot_new_projects = gdot_projects[~gdot_projects['Project_ID'].isin(
    GDOT_master['Project_ID'])].copy()

# ✅ If this returns any rows, run them through subsequent web scraping functions
gdot_new_projects

,geometry,Project_ID,Project_counties,Is_TIA_project,Contractor_name,TPRO_PROJ_COMPLETE_DT,Project_description,TIME_STOPPED_DATE,SUBSTL_WORK_COMPL_DATE,CONSTRUCTION_STATUS_DERIVED,Last_refresh,Project_URL,DISTRICT
2142,LINESTRING (-84.95342806015991 32.401288769488...,0017138,Muscogee,No,None,NaN,Military Drive From Infantry Road To Hampton Inn,NaN,NaN,PRE-CONSTRUCTION,1743450930000,https://www.dot.ga.gov/applications/geopi/Page...,2
2603,LINESTRING (-84.83481602011275 33.925422302647...,0013760,Paulding,No,None,NaN,SR 6Bu/SR 61 @ CS 610/Legion Road,NaN,NaN,PRE-CONSTRUCTION,1743450930000,https://www.dot.ga.gov/applications/geopi/Page...,14
3619,LINESTRING (-83.93912793360182 32.560242848555...,M006739,Peach,No,None,NaN,SR 96 From SR 49 Conn To Vienna Street,NaN,NaN,PRE-CONSTRUCTION,1743450930000,https://www.dot.ga.gov/applications/geopi/Page...,2
3675,LINESTRING (-84.62353078935584 32.845091156827...,M006750,"Harris , Meriwether , Talbot",No,None,NaN,SR 190 From SR 1 To SR 41,NaN,NaN,PRE-CONSTRUCTION,1743450930000,https://www.dot.ga.gov/applications/geopi/Page...,3
5559,LINESTRING (-81.8046065446991 31.1485868456123...,0021225,"Brantley , Camden",No,None,NaN,New Post Road @ County Line Branch,NaN,NaN,PRE-CONSTRUCTION,1743450930000,https://www.dot.ga.gov/applications/geopi/Page...,1
5560,LINESTRING (-84.24810927599651 33.114177749203...,0021226,Pike,No,None,NaN,Line Road @ Turnpike Creek,NaN,NaN,PRE-CONSTRUCTION,1743450930000,https://www.dot.ga.gov/applications/geopi/Page...,3
5561,LINESTRING (-82.55894794890067 32.772379997362...,0021227,Johnson,No,None,NaN,New Home Church Road @ Little Ohoopee River,NaN,NaN,PRE-CONSTRUCTION,1743450930000,https://www.dot.ga.gov/applications/geopi/Page...,12
5562,"LINESTRING (-82.514879376856 33.7724741096696,...",0021228,Lincoln,No,None,NaN,Lewis Cook Road @ Florence Creek,NaN,NaN,PRE-CONSTRUCTION,1743450930000,https://www.dot.ga.gov/applications/geopi/Page...,12
5563,LINESTRING (-82.6413021119917 32.3968164430120...,0021229,Treutlen,No,None,NaN,Knox Mill Road @ Rocky Creek,NaN,NaN,PRE-CONSTRUCTION,1743450930000,https://www.dot.ga.gov/applications/geopi/Page...,12
5564,LINESTRING (-84.11691229875628 34.406819659952...,0021233,Dawson,No,None,NaN,SR 9 @ Perimeter Road,NaN,NaN,PRE-CONSTRUCTION,1743450930000,https://www.dot.ga.gov/applications/geopi/Page...,7


In [ ]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

# Input and output file paths
INPUT_CSV = "GDOT_export.csv"
OUTPUT_DIR = "scraped/"

districts = [
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14
]

# Read the input CSV once
df = gdot_new_projects.copy()


# ================================
# ✅ Scraping Function (Outside Loop)
# ================================
async def scrape_project_data(district, df):
    """Scrape project data for a given district and return a cleaned DataFrame."""
    df_district = df[df["DISTRICT"] == district]
    total_projects = len(df_district)

    # Skip if no projects in this district
    if total_projects == 0:
        print(f"⚠️ No projects found for District {district}. Skipping...")
        return None

    all_data = []
    project_index = 0

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        for index, row in df_district.iterrows():
            project_id = row["Project_ID"]
            project_url = row["Project_URL"]
            project_index += 1

            # Progress indicator
            percentage = (project_index / total_projects) * 100
            print(
                f"Scraping Project {project_id} in District {district} "
                f"({project_index} of {total_projects}, {percentage:.1f}% complete)"
            )

            # Open the project page
            try:
                await page.goto(project_url)
            except Exception as e:
                print(
                    f"❌ Error opening project page for Project ID {project_id}: {e}")
                continue

            # Wait for the table to load before scraping
            try:
                await page.wait_for_selector("table.rgMasterTable", timeout=15000)
            except Exception as e:
                print(f"❌ Table not found for Project ID {project_id}: {e}")
                # Set default values for scraped fields
                project_description = ""
                project_manager = ""
                completion_date = ""
                lower_table_data = []
            else:
                # Scrape Project Description
                project_description = await scrape_project_description(page)

                # Scrape Project Manager & Completion Date
                project_manager, completion_date = await scrape_project_info(page)

                # Scrape Lower Table Data
                lower_table_data = await scrape_lower_table(page)

            for row_data in lower_table_data:
                all_data.append(
                    [
                        project_id,
                        project_url,
                        *row_data,
                        project_description,
                        project_manager,
                        completion_date,
                    ]
                )

             # ✅ Add delay to avoid hitting the server too fast
            await asyncio.sleep(0.5)  # Wait 1 second between project scrapes

        await browser.close()

    # Return scraped data as DataFrame
    return clean_and_save_data(all_data, district)


# ================================
# ✅ Helper Functions
# ================================
async def scrape_project_description(page):
    """Scrape the project description from the description table."""
    try:
        description_rows = await page.query_selector_all("table.ProjectDescriptionTable tbody tr")
        if len(description_rows) >= 2:
            second_row_cells = await description_rows[1].query_selector_all("td")
            if second_row_cells:
                return (await second_row_cells[0].inner_text()).strip()
    except Exception:
        pass
    return "N/A"


async def scrape_project_info(page):
    """Scrape project manager and completion date from project info table."""
    try:
        project_info_rows = await page.query_selector_all("table.ProjectInformationTable tbody tr")
        if len(project_info_rows) >= 3:
            third_row_cells = await project_info_rows[2].query_selector_all("td")
            if len(third_row_cells) >= 4:
                project_manager = (await third_row_cells[1].inner_text()).strip()
                completion_date = (await third_row_cells[3].inner_text()).strip()
                return project_manager, completion_date
    except Exception:
        pass
    return "N/A", "N/A"


async def scrape_lower_table(page):
    """Scrape the lower table data from the page."""
    all_rows_data = []
    rows = await page.query_selector_all("table.rgMasterTable tbody tr")
    for row in rows:
        cells = await row.query_selector_all("td")
        cell_data = [await cell.inner_text() for cell in cells]
        clean_data = [cell.replace("\xa0", " ").replace(
            "¬†", "").strip() for cell in cell_data]
        if len(clean_data) == 4:
            all_rows_data.append(clean_data)
    return all_rows_data


def clean_and_save_data(all_data, district):
    """Clean and save scraped data to a CSV."""
    if not all_data:
        print(f"⚠️ No valid data for District {district}. Skipping save...")
        return None

    columns = [
        "Project_ID",
        "Project_URL",
        "Activity",
        "Program Year",
        "Cost Estimate",
        "Date of Last Estimate",
        "Project_description",
        "Project_manager",
        "Completion_date",
    ]

    clean_df = pd.DataFrame(all_data, columns=columns)
    clean_df = clean_df.replace("", pd.NA).dropna(
        how="all", subset=columns[2:])
    clean_df["Cost Estimate"] = (
        clean_df["Cost Estimate"].fillna(0).str.replace(
            "$", "").str.replace(",", "").astype(float)
    )

    final_df = clean_df.groupby("Project_ID", as_index=False).agg(
        Project_URL=("Project_URL", "first"),
        Total_Cost=("Cost Estimate", "sum"),
        Project_manager=("Project_manager", "first"),
        Description=("Project_description", "first"),
    )

    output_path = f"{OUTPUT_DIR}/GDOT_scraped_District{district}.csv"
    final_df.to_csv(output_path, index=False)
    print(
        f"✅ Scraping complete for District {district}. Data saved to: {output_path}")
    return final_df


# ================================
# 🚀 Main Function
# ================================
async def main():
    """Run scraping for all districts in parallel."""
    for district in districts:
        await scrape_project_data(district, df)

        # ✅ Add delay between districts
        print(f"⏳ Waiting before scraping the next district...")
        await asyncio.sleep(2)  # 5 seconds delay between districts

    print("✅ All districts scraped successfully!")


# Run the scraper
if __name__ == "__main__":
    await main()

Scraping Project 0015114 in District 12 (1 of 783, 0.1% complete)
Scraping Project 0015128 in District 12 (2 of 783, 0.3% complete)
Scraping Project 0017571 in District 12 (3 of 783, 0.4% complete)
Scraping Project 0017572 in District 12 (4 of 783, 0.5% complete)
Scraping Project 0017573 in District 12 (5 of 783, 0.6% complete)
Scraping Project 0017574 in District 12 (6 of 783, 0.8% complete)
Scraping Project 0017575 in District 12 (7 of 783, 0.9% complete)
Scraping Project 0017590 in District 12 (8 of 783, 1.0% complete)
Scraping Project 0017595 in District 12 (9 of 783, 1.1% complete)
Scraping Project 0017596 in District 12 (10 of 783, 1.3% complete)
Scraping Project 0017597 in District 12 (11 of 783, 1.4% complete)
Scraping Project 0017598 in District 12 (12 of 783, 1.5% complete)
Scraping Project 0017599 in District 12 (13 of 783, 1.7% complete)
Scraping Project 0017600 in District 12 (14 of 783, 1.8% complete)
Scraping Project 0017601 in District 12 (15 of 783, 1.9% complete)
Scra

### 3) Prepare & export data for GeoJSON export


In [8]:
# for now, don't include the new projects (will update later)
gdot_projects = gdot_projects[~gdot_projects['Project_ID'].isin(
    gdot_new_projects['Project_ID'])]

# pare down the API exported GeoDataFrame
gdot_projects_slimmed = gdot_projects[[
    'Project_ID',
    'Project_counties',
    'Project_description',
    'CONSTRUCTION_STATUS_DERIVED',
    'DISTRICT',
    'geometry',
]]

# merge with what we already have
gdot_joined = pd.merge(
    gdot_projects_slimmed,
    GDOT_master,
    how='left',
    on='Project_ID'
)

# drop any duplicate rows based on both 'Project_ID' and 'DISTRICT'
gdot_joined = gdot_joined.drop_duplicates(
    subset=['Project_ID', 'DISTRICT'], keep='first'
)

# export
gdot_export = gdot_joined.drop(columns=['geometry'])
gdot_export.to_csv(
    'GDOT_export_joined.csv', index=False)

# calculate summary statistics for each group
summary_stats = gdot_joined.groupby(['CONSTRUCTION_STATUS_DERIVED', 'DISTRICT']).agg(
    total_projects=('Total_Cost', 'count'),
    total_cost=('Total_Cost', 'sum'),
    average_cost=('Total_Cost', 'mean')
).reset_index()

# calculate the total projects for each Distict and add it as a row where 'CONSTRUCTION_STATUS_DERIVED' is 'Total'
total_projects = summary_stats.groupby('DISTRICT').agg(
    total_projects=('total_projects', 'sum'),
    total_cost=('total_cost', 'sum'),
).reset_index()

# calculate the average cost for each Distict and add it as a row where 'CONSTRUCTION_STATUS_DERIVED' is 'Total'
total_projects['average_cost'] = total_projects['total_cost'] / \
    total_projects['total_projects']
total_projects['CONSTRUCTION_STATUS_DERIVED'] = 'ALL'

# add the total projects row to the summary_stats DataFrame
summary_stats_ALL = pd.concat(
    [summary_stats, total_projects], ignore_index=True)

# ✅ Calculate statewide totals (DISTRICT = "ALL")
statewide_totals = summary_stats_ALL.groupby('CONSTRUCTION_STATUS_DERIVED').agg(
    total_projects=('total_projects', 'sum'),
    total_cost=('total_cost', 'sum'),
).reset_index()

# Calculate the average cost for statewide totals
statewide_totals['average_cost'] = statewide_totals['total_cost'] / \
    statewide_totals['total_projects']
statewide_totals['DISTRICT'] = 'ALL'

# Add statewide totals to the summary_stats_ALL DataFrame
summary_stats_ALL = pd.concat(
    [summary_stats_ALL, statewide_totals], ignore_index=True)

# Exporting the DataFrame to a JSON & CSV
summary_stats_ALL.to_csv('data/summary-stats.csv', index=False)
summary_stats_ALL.to_json('data/summary-stats.json',
                          orient='records', indent=4)

# Ensure all properties are JSON serializable
gdot_joined = gdot_joined.replace(
    {np.nan: None, np.inf: None, -np.inf: None})

# create a new GeoJSON feature collection with a unique integer ID for each feature
features = []
for i, row in gdot_joined.iterrows():

    feature = {
        'type': 'Feature',
        'id': i + 1,  # assign a unique integer ID
        'properties': row.drop('geometry').to_dict(),
        'geometry': row['geometry'].__geo_interface__
    }
    features.append(feature)

# create a new GeoJSON feature collection
collection = {
    'type': 'FeatureCollection',
    'features': features
}

# export the feature collection to a GeoJSON file
with open("data/GDOT_export.geojson", 'w') as f:
    json.dump(collection, f)

# print status
print(f"✅ Successfully exported {len(gdot_joined):,} records!")

# Display final GeoDataFrame
gdot_joined.sample(3)

✅ Successfully exported 3,994 records!


,Project_ID,Project_counties,Project_description,CONSTRUCTION_STATUS_DERIVED,DISTRICT,geometry,Project_URL,Total_Cost,Project_manager,Description
2935,0017939,Morgan,SR 83 @ Big Indian Creek,PRE-CONSTRUCTION,10,LINESTRING (-83.52639913226979 33.524037317912...,https://www.dot.ga.gov/applications/geopi/Page...,7219801.11,Benny Brock,Replacement of an existing 2-lane bridge Struc...
1784,0013921,Bibb,Tucker RD (CR 742) - Construction Of Bridges,UNDER-CONSTRUCTION,8,LINESTRING (-83.74654709041587 32.857986268212...,https://www.dot.ga.gov/applications/geopi/Page...,13922618.38,Victor Gill,This project will replace two bridges located ...
2381,0020145,Marion,James Rogers Road @ Hardage Ford Creek 4 MI N ...,PRE-CONSTRUCTION,2,LINESTRING (-84.51822025104435 32.380519281553...,https://www.dot.ga.gov/applications/geopi/Page...,7075000.00,Lawrence Brown,None


In [16]:
url_rc = "https://services1.arcgis.com/Ug5xGQbHsD8zuZzM/arcgis/rest/services/ACS_2023_Geographic_boundaries_view/FeatureServer/0/query?where=SumLev%20%3D%20%27RC%27&outFields=*&outSR=4326&f=json"

gdf_rcs = gpd.read_file(url_rc)
gdf_rcs = gdf_rcs.to_crs(epsg=4326)

# create a new geodataframe of centroids that only retains the geometry and 'Name' column
gdf_rcs_centroids = gdf_rcs.copy()
gdf_rcs_centroids['geometry'] = gdf_rcs_centroids['geometry'].centroid
gdf_rcs_centroids = gdf_rcs_centroids[['Name', 'geometry']]

gdf_rcs.to_file('data/RCs/RCs.geojson', driver='GeoJSON')
gdf_rcs_centroids.to_file('data/RCs/RCs_centroids.geojson', driver='GeoJSON')

/var/folders/yt/f4vdhy_56sg4y2gk8bwqbcvc0000gn/T/ipykernel_96099/1002902677.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_rcs_centroids['geometry'] = gdf_rcs_centroids['geometry'].centroid


In [17]:
url_2 = "https://services1.arcgis.com/Ug5xGQbHsD8zuZzM/arcgis/rest/services/ACS_2023_Geographic_boundaries_view/FeatureServer/0/query?where=SumLev%20%3D%20'AAA'&outFields=*&outSR=4326&f=json"
gdf_aaa = gpd.read_file(url_2)

# set the crs to 4326
gdf_aaa = gdf_aaa.to_crs(epsg=4326)

gdf_aaa_centroids = gdf_aaa.copy()
gdf_aaa_centroids['geometry'] = gdf_aaa_centroids['geometry'].centroid
gdf_aaa_centroids = gdf_aaa_centroids[['Name', 'geometry']]

gdf_aaa.to_file('data/AAA/aging.geojson', driver='GeoJSON')
gdf_aaa_centroids.to_file('data/AAA/aging_centroids.geojson', driver='GeoJSON')

/var/folders/yt/f4vdhy_56sg4y2gk8bwqbcvc0000gn/T/ipykernel_96099/3312558113.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_aaa_centroids['geometry'] = gdf_aaa_centroids['geometry'].centroid


In [30]:
workforce = gpd.read_file('data/Workforce/WF_GA.gpkg')

df = pd.read_csv('data/Workforce/ga_xwalk.csv')
df = df.drop(columns=[
    'st',
    'stusps',
    'stname',
    'createdate',
    'trctname',
    'cbsa',
    'cbsaname',
    'zcta',
    'zctaname',
    'stplc',
    'stplcname',
    'ctycsub',
    'ctycsubname',
    'tsub',
    'tsubname',
    'stanrc',
    'stseconname',
    'stsecon',
    'trib',
    'tribname',
    'stanrcname',
    'mil',
    'milname',
])

df

/var/folders/yt/f4vdhy_56sg4y2gk8bwqbcvc0000gn/T/ipykernel_96099/4070864636.py:3: DtypeWarning: Columns (27,28,29,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/Workforce/ga_xwalk.csv')


,tabblk2020,cty,ctyname,trct,bgrp,bgrpname,stcd118,stcd118name,stsldl,stsldlname,stsldu,stslduname,stschool,stschoolname,stwib,stwibname,blklatdd,blklondd
0,130019501001000,13001,"Appling County, GA",13001950100,130019501001,"1 (Tract 9501, Appling, GA)",1301,GA-01,13157,"State House District 157, GA",13019,"State Senate District 19, GA",1300060,"Appling County School District, GA",13000016,16 Heart of Georgia/Altamaha WIB,31.922666,-82.310762
1,130019501001001,13001,"Appling County, GA",13001950100,130019501001,"1 (Tract 9501, Appling, GA)",1301,GA-01,13157,"State House District 157, GA",13019,"State Senate District 19, GA",1300060,"Appling County School District, GA",13000016,16 Heart of Georgia/Altamaha WIB,31.907902,-82.331439
2,130019501001002,13001,"Appling County, GA",13001950100,130019501001,"1 (Tract 9501, Appling, GA)",1301,GA-01,13157,"State House District 157, GA",13019,"State Senate District 19, GA",1300060,"Appling County School District, GA",13000016,16 Heart of Georgia/Altamaha WIB,31.934496,-82.307636
3,130019501001003,13001,"Appling County, GA",13001950100,130019501001,"1 (Tract 9501, Appling, GA)",1301,GA-01,13157,"State House District 157, GA",13019,"State Senate District 19, GA",1300060,"Appling County School District, GA",13000016,16 Heart of Georgia/Altamaha WIB,31.942781,-82.310738
4,130019501001004,13001,"Appling County, GA",13001950100,130019501001,"1 (Tract 9501, Appling, GA)",1301,GA-01,13157,"State House District 157, GA",13019,"State Senate District 19, GA",1300060,"Appling County School District, GA",13000016,16 Heart of Georgia/Altamaha WIB,31.939166,-82.354060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232712,133219506002054,13321,"Worth County, GA",13321950600,133219506002,"2 (Tract 9506, Worth, GA)",1308,GA-08,13152,"State House District 152, GA",13013,"State Senate District 13, GA",1305850,"Worth County School District, GA",13000017,17 Southwest Georgia WIB,31.339069,-83.850283
232713,133219506002055,13321,"Worth County, GA",13321950600,133219506002,"2 (Tract 9506, Worth, GA)",1308,GA-08,13152,"State House District 152, GA",13013,"State Senate District 13, GA",1305850,"Worth County School District, GA",13000017,17 Southwest Georgia WIB,31.334985,-83.833652
232714,133219506002056,13321,"Worth County, GA",13321950600,133219506002,"2 (Tract 9506, Worth, GA)",1308,GA-08,13152,"State House District 152, GA",13013,"State Senate District 13, GA",1305850,"Worth County School District, GA",13000017,17 Southwest Georgia WIB,31.336128,-83.879456
232715,133219506002057,13321,"Worth County, GA",13321950600,133219506002,"2 (Tract 9506, Worth, GA)",1308,GA-08,13152,"State House District 152, GA",13013,"State Senate District 13, GA",1305850,"Worth County School District, GA",13000017,17 Southwest Georgia WIB,31.334008,-83.884083
